In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [1]:
import triton_python_backend_utils as pb_utils

In [2]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [3]:
import pandas as pd
import numpy as np
import json

In [67]:
amount = np.array([['$45.23']]).astype('object')

In [68]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [69]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [70]:
model_name = "preprocessing"

In [71]:
inputs = [prepare_tensor('AMOUNT', amount)]

In [72]:
outputs = [
    httpclient.InferRequestedOutput("AMOUNT")
]

In [84]:
response = client.infer(model_name,
                        inputs)

In [85]:
output0_data = response.as_numpy("AMOUNT")

In [86]:
print(output0_data)

[45.23]
